Export data to grive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import date
from google.colab import files
from google.colab import drive
import shutil
import csv
import pickle

In [ ]:
#adjusting the layout
#import matplotlib.pyplot as plt

# General data processing

The model that we want to make is spatiotemporal geostrophic current prediction. The geostrophic current equations and its discrete representation are showed bellow:

${\displaystyle v=\frac{g}{f}\frac{\partial ADT}{\partial x} \approx \frac{g}{h} (\frac{ADT_{east}-ADT_{west}}{\Delta x}) }$

${\displaystyle u=-\frac{g}{f}\frac{\partial ADT}{\partial x} \approx \frac{g}{h} (\frac{ADT_{north}-ADT_{south}}{\Delta y}) }$



*   ADT: absolute dynamic topography
*   x: longitude
*   y: latitude
*   f: coriolis parameter
*   g: gravity acceleration
*   u: longitudidinal geostrohic current
*   v: latitudinal geostrohic current

Based on those equations we need to get the neighbouring value of each cells especially for ADT data

In [ ]:
#Olah data train
center  = []
south   = []
north   = []
east    = []
west    = []

u_geos  = []
v_geos  = []
for year in range(2013,2022): #nanti 2019 2022
  for month in range(1,13):
    for day in range(1,32):
      try:
        #ambil data nc
        data  = xr.open_dataset('/content/drive/MyDrive/Tugas Akhir/DATAFIX/dt_global_twosat_phy_l4_'+"%04d" % (year)+"%02d" % (month)+"%02d" % (day)+'_vDT2021.nc')
        data  = data.where((data['latitude'] > -13.65) & (data['latitude'] < 9.5) & (data['longitude'] > 93.6) & (data['longitude'] < 142.7), drop=True)


        center.append(data['adt'].to_numpy().reshape(93,197)[1:-1,1:-1])
        south.append(np.roll(data['adt'].to_numpy(), 1,axis=1).reshape(93,197)[1:-1,1:-1])
        north.append(np.roll(data['adt'].to_numpy(),-1,axis=1).reshape(93,197)[1:-1,1:-1])
        west.append(np.roll(data['adt'].to_numpy(), 1,axis=2).reshape(93,197)[1:-1,1:-1])
        east.append(np.roll(data['adt'].to_numpy(),-1,axis=2).reshape(93,197)[1:-1,1:-1])

        u_geos.append(data['ugos'].to_numpy().reshape(93,197)[1:-1,1:-1])
        v_geos.append(data['vgos'].to_numpy().reshape(93,197)[1:-1,1:-1])

      except:
        print('No such file or directory for year :'+str(year)+" month :"+str(month)+" day :"+str(day))
        break

No such file or directory for year :2013 month :2 day :29
No such file or directory for year :2013 month :4 day :31
No such file or directory for year :2013 month :6 day :31
No such file or directory for year :2013 month :9 day :31
No such file or directory for year :2013 month :11 day :31
No such file or directory for year :2014 month :2 day :29
No such file or directory for year :2014 month :4 day :31
No such file or directory for year :2014 month :6 day :31
No such file or directory for year :2014 month :9 day :31
No such file or directory for year :2014 month :11 day :31
No such file or directory for year :2015 month :2 day :29
No such file or directory for year :2015 month :4 day :31
No such file or directory for year :2015 month :6 day :31
No such file or directory for year :2015 month :9 day :31
No such file or directory for year :2015 month :11 day :31
No such file or directory for year :2016 month :2 day :30
No such file or directory for year :2016 month :4 day :31
No such fil

In [ ]:
import pickle
with open('/content/drive/MyDrive/Tugas Akhir/raw_data.pickle','wb') as w:
  pickle.dump({'center':center,
          'south':south,
          'north':north,
          'east':east,
          'west':west,
          'u_geos':u_geos,
          'v_geos':v_geos}, w, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# {'center':center,
#           'south':south,
#           'north':north,
#           'east':east,
#           'west':west,
#           'u_geos':u_geos,
#           'v_geos':v_geos}

{'center': [array([[1.136 , 1.117 , 1.0997, ...,    nan,    nan,    nan],
         [1.1161, 1.0992, 1.0857, ...,    nan,    nan,    nan],
         [1.0966, 1.0831, 1.0705, ...,    nan,    nan,    nan],
         ...,
         [1.0145, 1.0277, 1.0425, ..., 1.0969, 1.0974, 1.0938],
         [1.0044, 1.0155, 1.0297, ..., 1.1124, 1.1125, 1.1139],
         [0.9975, 1.0037, 1.0186, ..., 1.1217, 1.1255, 1.1278]]),
  array([[1.1172, 1.1   , 1.0844, ...,    nan,    nan,    nan],
         [1.1019, 1.0904, 1.0762, ...,    nan,    nan,    nan],
         [1.0849, 1.0727, 1.0612, ...,    nan,    nan,    nan],
         ...,
         [1.0156, 1.0283, 1.0424, ..., 1.0959, 1.0938, 1.0945],
         [1.0052, 1.017 , 1.0285, ..., 1.1095, 1.1093, 1.1087],
         [0.9999, 1.0097, 1.0208, ..., 1.1221, 1.1213, 1.1222]]),
  array([[1.101 , 1.0853, 1.0714, ...,    nan,    nan,    nan],
         [1.086 , 1.0723, 1.0618, ...,    nan,    nan,    nan],
         [1.071 , 1.0584, 1.05  , ...,    nan,    nan,    nan]